# Chapter 7 - Testing TinyML on Emulated Devices with Zephyr OS

In [ ]:
%tensorflow_version 2.x

### Python libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from tensorflow.keras import datasets, layers, models

### Constants

In [ ]:
TF_MODEL="cifar10"
TFL_MODEL_FILE="cifar10.tflite"

## Designing and training a tiny CIFAR-10 model

### Download the CIFAR-10 dataset

In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = datasets.cifar10.load_data()

### Normalize the pixel values between 0 and 1

In [ ]:
train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0

### Define a Python function to implement DWSC

In [ ]:
def separable_conv(input, ch, idx):
    x = layers.DepthwiseConv2D((3,3), padding="same", name='dwc0_dwsc'+str(idx))(input)
    x = layers.BatchNormalization( name='bn0_dwsc'+str(idx))(x)
    x = layers.Activation("relu", name='act0_dwsc'+str(idx))(x)
    x = layers.Conv2D(ch, (1,1), padding="same", name='conv0_dwsc'+str(idx))(x)
    x = layers.BatchNormalization(name='bn1_dwsc'+str(idx))(x)
    return layers.Activation("relu", name='act1_dwsc'+str(idx))(x)

### Design the convolution base

In [ ]:
input = layers.Input((32,32,3))
x = layers.Conv2D(16, (3, 3), padding='same', name='conv1')(input)
x = layers.BatchNormalization(name='bn1')(x)
x = layers.Activation("relu", name='act1')(x)
x = separable_conv(x, 24, 2)
x = layers.MaxPooling2D((2, 2), name='pool1')(x)
x = separable_conv(x, 48, 3)
x = layers.MaxPooling2D((2, 2), name='pool2')(x)
x = separable_conv(x, 96, 4)
x = separable_conv(x, 192, 5)
x = layers.MaxPooling2D((2, 2), name='pool3')(x)

### Design the classification head

In [ ]:
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(10)(x)

### Generate the model and print its summary

In [ ]:
model = Model(input, x)
model.summary()

### (Optional) Evaluate the tensor size for the intermediate tensors

In [ ]:
fig = plt.figure(dpi=100)

ax = fig.add_axes([0,0,1,1])

l_idx   = []
l_sizes = []

for layer in model.layers[1:]:
  shape = layer.output_shape
  shape = np.delete(shape, 0)
  size  = np.prod(shape)
  l_idx   = np.append(l_idx, layer.name)
  l_sizes = np.append(l_sizes, size)

ax.bar(l_idx, l_sizes)
plt.xticks(rotation='vertical')
plt.show()

### Compile and train the model with 10 epochs

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_imgs, train_lbls, epochs=10, 
                    validation_data=(test_imgs, test_lbls))

### Save the TensorFlow model as SavedModel

In [ ]:
model.save(TF_MODEL)

## Evaluating the accuracy of the TFLite model

### Select a few hundred of samples from the train dataset to calibrate the quantization

In [ ]:
cifar_ds = tf.data.Dataset.from_tensor_slices(train_imgs).batch(1)
def representative_data_gen():
  for i_value in cifar_ds.take(100):
    i_value_f32 = tf.dtypes.cast(i_value, tf.float32)
    yield [i_value_f32]

### Initialize the TFLite converter to perform the 8-bit quantization

In [ ]:
tfl_conv = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
tfl_conv.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
tfl_conv.optimizations = [tf.lite.Optimize.DEFAULT]
tfl_conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tfl_conv.inference_input_type = tf.int8
tfl_conv.inference_output_type = tf.int8

### Convert the model to TFLite file format and save it as .tflite

In [ ]:
tfl_model = tfl_conv.convert()
open(TFL_MODEL_FILE, "wb").write(tfl_model)

### Evaluate the model size

In [ ]:
print(len(tfl_model))

### Evaluate the accuracy of the quantized model using the test dataset

In [ ]:
# Initialize the TFLite interpreter
tfl_inter = tf.lite.Interpreter(model_content=tfl_model)

# Allocate the tensors
tfl_inter.allocate_tensors()

# Get input/output layer information
i_details = tfl_inter.get_input_details()[0]
o_details = tfl_inter.get_output_details()[0]

i_quant = i_details["quantization_parameters"]
o_quant = o_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]
o_scale      = o_quant['scales'][0]
o_zero_point = o_quant['zero_points'][0]

def classify(i_data, o_value):
  input_data = i_value.reshape((1, 32, 32, 3))
  i_value_f32 = tf.dtypes.cast(input_data, tf.float32)
  
  # Quantize (float -> 8-bit) the input (check if input layer is 8-bit, first)
  i_value_f32 = i_value_f32 / i_scale + i_zero_point
  i_value_s8 = tf.cast(i_value_f32, dtype=tf.int8)

  tfl_inter.set_tensor(i_details["index"], i_value_s8)
  tfl_inter.invoke()
  o_pred = tfl_inter.get_tensor(o_details["index"])[0]

  return (o_pred - o_zero_point) * o_scale

num_correct_samples = 0
num_total_samples   = len(list(test_imgs))

for i_value, o_value in zip(test_imgs, test_lbls):
  o_pred_f32 = classify(i_value, o_value)
  if np.argmax(o_pred_f32) == o_value:
    num_correct_samples += 1

### Print the accuracy of the quantized TFLite model

In [ ]:
print("Accuracy:", num_correct_samples/num_total_samples)

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i cifar10.tflite > model.cc

## Converting a NumPy image to a C-byte array

### Write a function to convert a 1D NumPy array of np.int8 values into a single string of integer values comma-separated

In [ ]:
def array_to_str(data):
    NUM_COLS = 12
    val_string = ''
    for i, val in enumerate(data):
        val_string += str(val)

        if (i + 1) < len(data):
            val_string += ','
        if (i + 1) % NUM_COLS == 0:
            val_string += '\n'
    return val_string

### Write a function to generate a C header file containing the input image stored in an int8_t array

In [ ]:
def gen_h_file(size, data, ilabel):
  str_out = f'int8_t g_test[] = '
  str_out += "\n{\n"
  str_out += f'{data}'
  str_out += '};\n'
  str_out += f"const int g_test_len = {size};\n"
  str_out += f"const int g_test_ilabel = {ilabel};\n"
  return str_out

### Create a Pandas data frame from the CIFAR-10 test dataset

In [ ]:
imgs = list(zip(test_imgs, test_lbls))
cols = ['Image', 'Label']
df = pd.DataFrame(imgs, columns = cols) 

### Get only ship images from the Pandas data frame

In [ ]:
cond = df['Label'] == 8
ship_samples = df[cond]

### Iterate over the ship images and run the inference. Convert the image that returns "ship" to C-byte array

In [ ]:
c_code = ""

for index, row in ship_samples.iterrows():
  i_value = np.asarray(row['Image'].tolist())
  o_value = np.asarray(row['Label'].tolist())
  o_pred_f32 = classify(i_value, o_value)
  if np.argmax(o_pred_f32) == o_value:
    i_value_f32 = i_value / i_scale + i_zero_point
    i_value_s8  = i_value_f32.astype(dtype=np.uint8)
    i_value_s8  = i_value_s8.ravel()

    # Generate a string from NumPy array
    val_string = array_to_str(i_value_s8)

    c_code = gen_h_file(i_value_s8.size, val_string, "8")
    break

### Save the generated image in the input.h file

In [ ]:
with open("input.h", 'w') as file:
  file.write(c_code)